Setup Instructions
==========

These instructions install `miniconda` and the requisite libraries for the tutorial in a `conda` environment, which is an indepenent python installation.

Jupyter has the similar concept of _kernels_, which are independent execution environments. They don't even have to be Python, kernels for other languages exist as well.

By loading a separate kernel for each project, we avoid the complication of different components/projects having weird interactions, ultimately helping reproducibility.

We first produce a new conda environment with the libraries we require, then we tell Jupyter about this new environment with the ipython executable

NOTE: This step may take a little while

In [ ]:
%%bash
#!/bin/bash

KERNEL_NAME="machine-learning-hats-2021"

# Download miniconda3 install script
if [[ ! -f $HOME/miniconda3.sh ]]; then
    wget --progress=dot:giga https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O $HOME/miniconda3.sh 2>&1 || { echo "Failed to download the miniconda3 installation script."; exit $?; }
fi

# Install miniconda3
if [[ ! -d $HOME/miniconda3 ]]; then
    echo "Installing miniconda3..."
    bash $HOME/miniconda3.sh -b -f -u -p $HOME/miniconda3 2>&1 || { echo "Failed to install miniconda3."; exit $?; }
fi
echo "Sourcing the miniconda3 setup script..."
source $HOME/miniconda3/etc/profile.d/conda.sh

# Update miniconda3 to the latest version
echo "Updating the miniconda3 release..."
conda update -y -n base -c defaults conda 2>&1 || { echo "Failed to properly update miniconda3."; exit $?; }

# Create conda environment
if [[ ! -d $HOME/miniconda3/envs/machine-learning-hats-2021 ]]; then
    echo "Installing mamba..."
    conda install -y mamba -n base -c conda-forge
    echo "Creating the miniconda3 environment \"$KERNEL_NAME\"..."
    mamba env create -f environment.yml --name $KERNEL_NAME || { echo "Failed to properly setup the environment \"$KERNEL_NAME\" using mamba."; exit $?; }
fi
echo "Activating the kernel \"$KERNEL_NAME\"..."
conda activate $KERNEL_NAME || { echo "Failed to activate the kernel \"$KERNEL_NAME\"."; exit $?; }

set -e

# Make a wrapper script to load miniconda python
if [[ ! -f python_wrapper.sh ]]; then
    echo "Creating the python_wrapper.sh script..."
    cat << 'EOF' > python_wrapper.sh
#!/bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate machine-learning-hats-2021
exec python "$@"
EOF
    chmod +x python_wrapper.sh
fi

# Create the kernel
if [[ ! -d $HOME/.local/share/jupyter/kernels/$KERNEL_NAME ]]; then
    echo "Creating the kernel $KERNEL_NAME..."
    mkdir -p "$HOME/.local/share/jupyter/kernels/$KERNEL_NAME"
    cat << EOF > "$HOME/.local/share/jupyter/kernels/$KERNEL_NAME/kernel.json"
{
 "display_name": "$KERNEL_NAME", 
 "language": "python", 
 "argv": [
  "$PWD/python_wrapper.sh", 
  "-m", 
  "ipykernel_launcher", 
  "-f", 
  "{connection_file}"
 ]
}
EOF
fi

# Report OK
echo "Loaded $KERNEL_NAME!"

Results
=======

If successful, you should see something similar to the following:

```
Loaded machine-learning-hats-2021!
```

The new kernel you just made will then show up in the various Jupyter after refreshing the page.